# Construyendo una red neural en Pytorch

Pytorch es una excelente herramienta para crear y entrenar modelos con diferentes arquitecturas.
Por lo mismo, antes de que se desarrollara Pytorch, un podía construir redes neurales con simplemente
paquetes básicos como `Numpy` y `math`.

En la siguiente sección, veremos como desarrollar una red neural simple con los simples paquetes de Python.

## Ejemplo de red neural con simplemente usando `Numpy` y Pytorch

Podemos entrenar un modelo simple con la función:

$$y = a + bx$$

Esta función consta de 3 inputs, y nosotros queremos utilizar `Numpy` para encontrar la relación entre
`x` e `y` utilizando la función de arriba.

Para empezar, importamos nuestros módulos y creamos data aleatorea con cierto ruido:

In [ ]:
# Módulos
%matplotlib inline
import numpy as np
import math
from sklearn import datasets
import matplotlib.pyplot as plt
import torch
import torch.nn as nn

random_state = 20

Y antes de cualquier cosa, inicializamos la semilla aleatorea:

In [ ]:
np.random.seed(random_state)

Y ahora creamos una data aleatoria para el *input* y *output* del modelo:

In [ ]:
# Definimos los parámetros
a_original = 10
b_original = 0.5

# Y luego creamos la data
X_numpy = np.linspace(-10, 10, 2000)
y_numpy = a_original + b_original * X_numpy

In [ ]:
#X_numpy, y_numpy, coef = datasets.make_regression(n_samples=1000, n_features=1, noise=5, random_state=random_state, coef=True)

Y podemos ver esta data:

In [ ]:
fig = plt.figure(figsize=(8,8), facecolor="white")
ax = fig.add_subplot(111)
ax.scatter(X_numpy, y_numpy, c="blue")
ax.set_aspect('equal')
ax.set_xlabel("X")
ax.set_ylabel("Y")

Lo que queremos lograr es que nuestro modelo pueda aprender la relación de la data y ver que tan bién puede
representar la función lineal de esta data. Los pasos a seguir son los siguientes:

1. Inicializar la pesas aleatoreamente
2. Especificar parámetros como la *velocidad de aprendizaje* (learning rate) del modelo, el número de épocas, etc.
3. Calcular la predicción del modelo
4. Comparar con la verdad y calcular la función de pérdida
5. Empezar con el "backpropagation", i.e. calculando las derivadas de la función de perdida con respecto a los coeficientes de la función.
6. Modificar las pesas y empezar de nuevo

In [ ]:
# Inicializando pesas aleatoreas
a_weight = np.random.randn()
b_weight = np.random.randn()

print(f">>> a: `{a_weight}` | b: `{a_weight}`")

Luego el valor de la velocidad de aprendizaje (learning rate) y el número de épocas:

In [ ]:
# Learning rate
learning_rate = 1e-6

# Número de épocas:
n_epoch = 2000

Y ahora estamos listos para empezar con el entrenamiento del modelo:

In [ ]:
# Iterando sobre cada época
for epoch_ii in range(n_epoch):
    # Iniciamos con el "forward pass" para predecir `y_pred`
    y_pred = a_weight + b_weight * X_numpy
    
    # Luego calculamos la función de pérdida
    # En este caso, la definimos como SUM[(y_pred - y_true)^2]
    # Esta función es la que se tratará de minimizar en cada iteración
    loss_val = np.sum(np.square(y_pred - y_numpy))
    #
    if epoch_ii % 200 == 0:
        print("Epoch: {0} | Loss: {1}".format(epoch_ii, loss_val))
    
    # Y ahora podemos ajustar las pesas con respecto a la función de pérdida
    # Esto es lo mismo que:
    #     d(loss)/d(y_pred) = d((y_pred - y_true)^2) = 2 * (y_pred - y)
    grad_y_pred = 2 * (y_pred - y_numpy)
    # Y ahora calculamos la derivada de las variables referente a la función de pérdida
    grad_a = grad_y_pred.sum()
    grad_b = (grad_y_pred * X_numpy).sum()
    
    # Y por último podemos ahora actualizar las pesas y empezar de nuevo
    a_weight -= learning_rate * grad_a
    b_weight -= learning_rate * grad_b

Y ahora podemos visualizar la comparar los valores que obtuvimos con los que
son en realidad los valores correctos:

In [ ]:
# Ya que ha terminado de aprender este "modelo", podemos verificar los
# resultados:

print(f">>> Predicción: y = {a_weight} + {b_weight} x")
print(f">>> Realidad  : y = {a_original} + {b_original} x")

Esto muestra que uno puede utilizar estos simples métodos para encontrar los coeficientes
de una función, que en este caso es linear.

### Mismo modelo pero en Pytorch

Ahora podemos hacer el mismo procedimiento, pero utilizando Pytorch para esto:

In [ ]:
# Inicializamos los tensores de entrada y salida
X_tensor = torch.from_numpy(X_numpy.astype(np.float32))
y_tensor = torch.from_numpy(y_numpy.astype(np.float32))

# Y modificamos la forma de `y_tensor`
X_tensor = X_tensor.view(y_tensor.shape[0], 1)
y_tensor = y_tensor.view(y_tensor.shape[0], 1)

In [ ]:
print(X_tensor)

In [ ]:
print(y_tensor)

Ahora podemos definir las variables de elementos y de "features":

In [ ]:
n_samples, n_features = X_tensor.shape

print(f"Número of elementos: {n_samples} | Número de features: {n_features}")

Lo siguientes pasos son para:

1. Crear el modelo lineal
2. Definir los parámetros y criterios
3. Utilizar un optimizador
4. Y por último, entrenar el modelo por un cierto número de épocas

#### Modelo

El modelo es un modelo lineal:

In [ ]:
# Modelo linear: f = wx + b
input_size = n_features
output_size = 1

# Definimos el modelo acá
model = nn.Linear(input_size, output_size)

#### Parámetros del modelo

En esta parte, decidimos cuáles son los valores de los hyper-parámetros del modelo:

In [ ]:
# Velocidad de aprendizaje
learning_rate = 0.01

# Función de pérdida
criterion = nn.MSELoss()

# Optimizador
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

#### Entrenar el modelo

Ya por último, vamos a entrenar el modelo:

In [ ]:
num_epochs = 10
for epoch in range(num_epochs):
    # Forward pass and loss
    y_predicted = model(X_tensor)
    loss = criterion(y_predicted, y_tensor)
    
    # Backward pass and update
    loss.backward()
    optimizer.step()

    # zero grad before new step
    optimizer.zero_grad()

    if (epoch+1) % 10 == 0:
        print(f'epoch: {epoch+1}, loss = {loss.item():.4f}')

#### Verificación de los datos

Es en esta parte donde podemos ver que tan bién se ejecutó el entrenamiento del modelo:

In [ ]:
# Plot
predicted = model(X_tensor).detach().numpy()

predicted

In [ ]:
plt.plot(X_numpy, y_numpy, 'ro')
plt.plot(X_numpy, predicted, 'b')
plt.show()